In [1]:
import pandas as pd
import re
import numpy as np
import json

In [2]:
def funcion1(row):
  lista = ['FUNCIONAMIENTO', 'DEUDA', 'INVERSION', 'NACIONAL']
  if pd.isna(row['CTA\nPROG']):
    return np.nan
  else:
    for i in lista:
      if i in row['CTA\nPROG']:
        return i


In [3]:
df_cleaned = {}
dic = {'A.  PRESUPUESTO DE FUNCIONAMIENTO ': 'FUNCIONAMIENTO',
 'B.  PRESUPUESTO DE SERVICIO DE LA DEUDA PÚBLICA': 'DEUDA',
 'C.  PRESUPUESTO DE INVERSION': 'INVERSION',
       'TOTAL PRESUPUESTO NACIONAL': 'TOTAL'}


for i in range(2013, 2023):
  # importar los datos saltando 4 renglones porque hay espacio nulo antes de la tabla
  df = pd.read_excel('data/Decreto_de_liquidacion.xlsx', sheet_name=f"Gastos PGN {i}",skiprows=4)
  # el número de renglones puede variar, si la columna CONCEPTO no se encuentra en las columnas de la importación, cambiar a 5 renglones

  if 'CONCEPTO' not in df.columns:
    df = pd.read_excel('data/Decreto_de_liquidacion.xlsx', sheet_name=f"Gastos PGN {i}",skiprows=5)

  # remover espacios de las columnas
  df.columns = [s.strip() for s in df.columns]

  # conservar únicamente la información cuando concepto o la cuenta no es nula
  df = df[~(df['CTA\nPROG'].isna()) | ~(df['CONCEPTO'].isna())]

  # crear un diccionario con las entidades y el código respectivo
  dic_entities = {}
  # iterar sobre los datos únicos de concepto (vienen en pares sección, entidad)
  for idx, valor in enumerate(df['CONCEPTO'].dropna().unique()):

    if idx % 2 == 0:
      # si el dato es par, se supone que hace referencia a sección, toma los cuatro dígitos
      val = re.findall(f"\d+", valor)[0]
      # agrega al diccionario una llave con los cuatro dígitos y ubica a la entidad que está inmediatamente después
      dic_entities[val] = df['CONCEPTO'].dropna().unique()[idx + 1]

  # filtrar para conservar solo estas tres columnas
  df = df.filter(['CTA\nPROG', 'CONCEPTO', 'TOTAL'])

  # extraer la palabra clave (funcionamiento, inversión o deuda)
  df['CTA\nPROG'] = df.apply(funcion1, axis=1)
  # reiniciar índice para evitar problemas de búsqueda
  df.reset_index(drop=True, inplace=True)
  # reversar el diccionario creado
  reversed_entities = {}
  for key, value in dic_entities.items():
    reversed_entities[value] = key

  # crear una columna para después agregar los códigos
  df['code'] = None

  # agregar el código y rellenar la columna concepto para normalizar la información
  for row in df.iterrows():
    val = row[1]['CONCEPTO']
    if pd.isna(val):
      df.loc[row[0], 'CONCEPTO'] = subs
      df.loc[row[0], 'code'] = code
    elif "SECCION" not in val:
      subs = val
      code = reversed_entities[val]
  # eliminar la fila con el valor de SECCION: en concepto
  df = df[~(df['CONCEPTO'].str.contains('SECCION:'))].reset_index(drop=True)

  # eliminar los datos nulos en el subset de total
  df = df.dropna().reset_index(drop=True)

  # agregar columna de código de sector
  df['sector_code'] = df['code'].str.extract(pat="(^\d{2})")
  df.rename(columns={'CTA\nPROG': 'cuenta',
                     'CONCEPTO': 'entidad',
                     'TOTAL': 'cop',
                     'code': 'entity_code'}, inplace=True)

  # agregar el dataframe al diccionario
  df_cleaned[i] = df


In [4]:
data = (pd
 .concat(df_cleaned.values(), keys=df_cleaned.keys())
 .reset_index()
 .drop(columns='level_1')
 .rename(columns={'level_0':'year'}))

In [5]:
data.to_csv('data/gastos.csv', index=False)

## Extraer datos 2023

In [6]:
df = pd.read_excel('data/ANEXO DECRETO 2023 vr.xlsx', skiprows=2, sheet_name='Anexo del Decreto PGN 2023')
df.columns = [i.strip() for i in df.columns]
lista = ['A. FUNCIONAMIENTO','C. INVERSIÓN', 'C. INVERSION', r'^TOTAL', "B. SERVICIO DE LA DEUDA PÚBLICA"]
df['CONCEPTO'] = df['CONCEPTO'].str.strip()

df['CTA\nPROG'] = df[(df['CONCEPTO'].str.contains('|'.join(lista)).fillna(False))]['CONCEPTO']
df['CONCEPTO'] = df[~(df['CONCEPTO'].str.contains('|'.join(lista)).fillna(False))]['CONCEPTO']
df = df.filter(['CTA\nPROG', 'CONCEPTO', 'TOTAL'])

df = df[~(df['CTA\nPROG'].isna()) | ~(df['CONCEPTO'].isna())]
df = df[~(df['CONCEPTO'].str.contains('UNIDAD:').fillna(False))]
df.reset_index(drop=True, inplace=True)
df
idxs = {}
for idx, s in enumerate(df['CONCEPTO']):
  if not pd.isna(s):
    if "SECCIÓN: " in s:
      idxs[idx] = s
      idxs[idx + 1] = df['CONCEPTO'][idx + 1]

df['CONCEPTO'] = df.iloc[list(idxs.keys())]['CONCEPTO']
df = df[~(df['CTA\nPROG'].isna()) | ~(df['CONCEPTO'].isna())]
df = df[df['CTA\nPROG']!='TOTAL PRESUPUESTO']

def funcion1(row):
  lista = ['FUNCIONAMIENTO', 'DEUDA', 'INVERSIÓN', 'NACIONAL']
  if pd.isna(row['CTA\nPROG']):
    return np.nan
  else:
    for i in lista:
      if i in row['CTA\nPROG']:
        return i

df['CTA\nPROG'] = df.apply(funcion1, axis=1)

dic_entities = {}
# iterar sobre los datos únicos de concepto (vienen en pares sección, entidad)

for idx, valor in enumerate(df['CONCEPTO'].dropna().unique()):

    if idx % 2 == 0:
      # si el dato es par, se supone que hace referencia a sección, toma los cuatro dígitos
      val = re.findall(f"\d+", valor)[0]
      # agrega al diccionario una llave con los cuatro dígitos y ubica a la entidad que está inmediatamente después
      dic_entities[val] = df['CONCEPTO'].dropna().unique()[idx + 1]


df.reset_index(drop=True, inplace=True)
  # reversar el diccionario creado
reversed_entities = {}
for key, value in dic_entities.items():
    reversed_entities[value] = key

  # crear una columna para después agregar los códigos
df['code'] = None

  # agregar el código y rellenar la columna concepto para normalizar la información
for row in df.iterrows():
    val = row[1]['CONCEPTO']
    if pd.isna(val):
      df.loc[row[0], 'CONCEPTO'] = subs
      df.loc[row[0], 'code'] = code
    elif "SECCIÓN" not in val:
      subs = val
      code = reversed_entities[val]
  # eliminar la fila con el valor de SECCION: en concepto
df = df[~(df['CONCEPTO'].str.contains('SECCION:'))].reset_index(drop=True)

  # eliminar los datos nulos en el subset de total
df = df.dropna().reset_index(drop=True)

  # agregar columna de código de sector
df['sector_code'] = df['code'].str.extract(pat="(^\d{2})")
df.rename(columns={'CTA\nPROG': 'cuenta',
                     'CONCEPTO': 'entidad',
                     'TOTAL': 'cop',
                     'code': 'entity_code'}, inplace=True)

In [7]:
ads = pd.read_excel('data/Decreto de Liquidaci_n 1234 de 2023 (1).xlsx',
              sheet_name='Art. 2 Adiciones PGN 2023',
              skiprows=3)
ads.columns = ads.columns.str.strip()
ads = (ads.rename(columns = {'CONCEPTO':'nans',
             'Unnamed: 3':'CONCEPTO'})
     .filter(['CTA', 'CONCEPTO', 'TOTAL']))

# mantener únicamente las cuentas y total final
lista = ['INVERSIÓN', 'TOTAL', 'FUNCIONAMIENTO']
ads['CTA'] = ads[(ads['CTA'].str.contains('|'.join(lista)).fillna(False))]['CTA']
ads['CTA'] = ads[~(ads['CTA'].str.contains('SECCIÓN').fillna(False))]['CTA']
#eliminar cuando sea nan tanto en cta como en concepto
ads = ads[~(ads['CTA'].isna()) | ~(ads['CONCEPTO'].isna())]
ads
#cambiar nombres de las cuentas
def funcion1(row):
  lista = ['FUNCIONAMIENTO', 'DEUDA', 'INVERSIÓN', 'TOTAL']
  if pd.isna(row['CTA']):
    return np.nan
  else:
    for i in lista:
      if i in row['CTA']:
        return i

ads['CTA'] = ads.apply(funcion1, axis=1)
ads.reset_index(drop=True, inplace=True)


#extraer sector
dic_entities = {}
# iterar sobre los datos únicos de concepto (vienen en pares sección, entidad)
for idx, valor in enumerate(ads['CONCEPTO'].dropna().unique()):

    if idx % 2 == 0:
      # si el dato es par, se supone que hace referencia a sección, toma los cuatro dígitos
      val = re.findall(f"\d+", valor)[0]
      # agrega al diccionario una llave con los cuatro dígitos y ubica a la entidad que está inmediatamente después
      dic_entities[val] = ads['CONCEPTO'].dropna().unique()[idx + 1]

ads.reset_index(drop=True, inplace=True)
  # reversar el diccionario creado
reversed_entities = {}
for key, value in dic_entities.items():
    reversed_entities[value] = key

  # crear una columna para después agregar los códigos
ads['code'] = np.nan

  # agregar el código y rellenar la columna concepto para normalizar la información
for row in ads.iterrows():

    val = row[1]['CONCEPTO']
    if pd.isna(val):
      ads.loc[row[0], 'CONCEPTO'] = subs
      ads.loc[row[0], 'code'] = code
    elif "SECCIÓN" not in val:
      subs = val
      code = reversed_entities[val]

ads.rename(columns={'CTA': 'cuenta',
                     'CONCEPTO': 'entidad',
                     'TOTAL': 'adicion',
                     'code': 'entity_code'}, inplace=True)

ads = ads[ads['entity_code'].notnull()]
ads.reset_index(drop=True, inplace=True)

/var/folders/rd/zs4xvzqn0qqds7psf6_1f74m0000gn/T/ipykernel_6349/1259952354.py:56: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0101' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ads.loc[row[0], 'code'] = code


In [8]:
ads['adicion'] = ads['adicion'].astype(int)

In [9]:
reds = pd.read_excel('data/Decreto de Liquidaci_n 1234 de 2023 (1).xlsx',
            sheet_name='Art. 3 Reducciones PGN 2023',
            skiprows=4)

reds = (reds.rename(columns = {'CONCEPTO':'nans',
             'Unnamed: 3':'CONCEPTO'})
     .filter(['CTA', 'CONCEPTO', 'TOTAL']))

#mantener únicamente las cuentas y total final
lista = ['INVERSIÓN', 'TOTAL', 'FUNCIONAMIENTO']
reds['CTA'] = reds[(reds['CTA'].str.contains('|'.join(lista)).fillna(False))]['CTA']
reds['CTA'] = reds[~(reds['CTA'].str.contains('SECCIÓN').fillna(False))]['CTA']
#eliminar cuando sea nan tanto en cta como en concepto
reds = reds[~(reds['CTA'].isna()) | ~(reds['CONCEPTO'].isna())]

#cambiar nombres de las cuentas
def funcion1(row):
  lista = ['FUNCIONAMIENTO', 'DEUDA', 'INVERSIÓN', 'TOTAL']
  if pd.isna(row['CTA']):
    return np.nan
  else:
    for i in lista:
      if i in row['CTA']:
        return i

reds['CTA'] = reds.apply(funcion1, axis=1)
reds.reset_index(drop=True, inplace=True)


#extraer sector
dic_entities = {}
# iterar sobre los datos únicos de concepto (vienen en pares sección, entidad)
for idx, valor in enumerate(reds['CONCEPTO'].dropna().unique()):

    if idx % 2 == 0:
      # si el dato es par, se supone que hace referencia a sección, toma los cuatro dígitos
      val = re.findall(f"\d+", valor)[0]
      # agrega al diccionario una llave con los cuatro dígitos y ubica a la entidad que está inmediatamente después
      dic_entities[val] = reds['CONCEPTO'].dropna().unique()[idx + 1]

reds.reset_index(drop=True, inplace=True)
  # reversar el diccionario creado
reversed_entities = {}
for key, value in dic_entities.items():
    reversed_entities[value] = key

  # crear una columna para después agregar los códigos
reds['code'] = np.nan

  # agregar el código y rellenar la columna concepto para normalizar la información
for row in reds.iterrows():

    val = row[1]['CONCEPTO']
    if pd.isna(val):
      reds.loc[row[0], 'CONCEPTO'] = subs
      reds.loc[row[0], 'code'] = code
    elif "SECCIÓN" not in val:
      subs = val
      code = reversed_entities[val]

reds.rename(columns={'CTA': 'cuenta',
                     'CONCEPTO': 'entidad',
                     'TOTAL': 'reduccion',
                     'code': 'entity_code'}, inplace=True)

reds = reds[reds['entity_code'].notnull()]
reds.reset_index(drop=True, inplace=True)
reds['reduccion'] = reds['reduccion'].astype(int)
reds.drop(index=13, inplace=True)
ads.drop(index=86, inplace=True)

/var/folders/rd/zs4xvzqn0qqds7psf6_1f74m0000gn/T/ipykernel_6349/4127525029.py:56: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0201' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  reds.loc[row[0], 'code'] = code


In [10]:
gastos = pd.read_csv('data/gastos.csv')

In [11]:
df['year'] = 2023

In [12]:
changes = pd.merge(reds, ads, on=['cuenta', 'entity_code', 'entidad'], how='outer').fillna(0)
changes['neto'] = changes['adicion'] - changes['reduccion']
changes['entidad'] = changes['entidad'].str.strip()
changes['year'] = 2023

pgn_2013 = df.merge(changes[['year', 'cuenta', 'entidad', 'entity_code','neto']], on=['year','cuenta', 'entidad', 'entity_code'], how='outer').fillna(0)
pgn_2013['cop'] = pgn_2013['cop'] + pgn_2013['neto']
pgn_2013.drop(columns='neto', inplace=True)



## Extraer datos 2024

In [13]:
df = pd.read_excel('data/Anexo Detalle de Gasto - Decreto PGN 2024.xlsx', skiprows=2)
df.columns = [i.strip() for i in df.columns]
lista = ['A. FUNCIONAMIENTO','C. INVERSIÓN', 'C. INVERSION', r'^TOTAL', "B. SERVICIO DE LA DEUDA PÚBLICA"]
df['CTA\nPROG'] = df[(df['CONCEPTO'].str.contains('|'.join(lista)).fillna(False))]['CONCEPTO']
df['CONCEPTO'] = df[~(df['CONCEPTO'].str.contains('|'.join(lista)).fillna(False))]['CONCEPTO']
df = df.filter(['CTA\nPROG', 'CONCEPTO', 'TOTAL'])

df = df[~(df['CTA\nPROG'].isna()) | ~(df['CONCEPTO'].isna())]
df = df[~(df['CONCEPTO'].str.contains('UNIDAD:').fillna(False))]
df.reset_index(drop=True, inplace=True)

idxs = {}
for idx, s in enumerate(df['CONCEPTO']):
  if not pd.isna(s):
    if "SECCIÓN: " in s:
      idxs[idx] = s
      idxs[idx + 1] = df['CONCEPTO'][idx + 1]

df['CONCEPTO'] = df.iloc[list(idxs.keys())]['CONCEPTO']
df = df[~(df['CTA\nPROG'].isna()) | ~(df['CONCEPTO'].isna())]
df = df[df['CTA\nPROG']!='TOTAL PRESUPUESTO']

def funcion1(row):
  lista = ['FUNCIONAMIENTO', 'DEUDA', 'INVERSIÓN', 'NACIONAL']
  if pd.isna(row['CTA\nPROG']):
    return np.nan
  else:
    for i in lista:
      if i in row['CTA\nPROG']:
        return i

df['CTA\nPROG'] = df.apply(funcion1, axis=1)

dic_entities = {}
# iterar sobre los datos únicos de concepto (vienen en pares sección, entidad)
for idx, valor in enumerate(df['CONCEPTO'].dropna().unique()):

    if idx % 2 == 0:
      # si el dato es par, se supone que hace referencia a sección, toma los cuatro dígitos
      val = re.findall(f"\d+", valor)[0]
      # agrega al diccionario una llave con los cuatro dígitos y ubica a la entidad que está inmediatamente después
      dic_entities[val] = df['CONCEPTO'].dropna().unique()[idx + 1]

df.reset_index(drop=True, inplace=True)
  # reversar el diccionario creado
reversed_entities = {}
for key, value in dic_entities.items():
    reversed_entities[value] = key

  # crear una columna para después agregar los códigos
df['code'] = None

  # agregar el código y rellenar la columna concepto para normalizar la información
for row in df.iterrows():
    val = row[1]['CONCEPTO']
    if pd.isna(val):
      df.loc[row[0], 'CONCEPTO'] = subs
      df.loc[row[0], 'code'] = code
    elif "SECCIÓN" not in val:
      subs = val
      code = reversed_entities[val]
  # eliminar la fila con el valor de SECCION: en concepto
df = df[~(df['CONCEPTO'].str.contains('SECCION:'))].reset_index(drop=True)

  # eliminar los datos nulos en el subset de total
df = df.dropna().reset_index(drop=True)

  # agregar columna de código de sector
df['sector_code'] = df['code'].str.extract(pat="(^\d{2})")
df.rename(columns={'CTA\nPROG': 'cuenta',
                     'CONCEPTO': 'entidad',
                     'TOTAL': 'cop',
                     'code': 'entity_code'}, inplace=True)

df['year'] = 2024

In [14]:
pd.concat([gastos, pgn_2013, df]).to_csv('data/gastos_2013-2024.csv', index=False)
final = pd.concat([gastos, pgn_2013, df])
final = final[final['cuenta'] != 'NACIONAL']
final.to_csv('data/gastos_2013-2024.csv', index=False)

## Deflactación

In [15]:
df = pd.read_csv('data/gastos_2013-2024.csv')
ipc = pd.read_csv('data/ipc.csv')

In [16]:
ipc.columns = ipc.columns.str.strip().str.lower().str.replace(" ","_")
ipc['year'] = pd.to_datetime(ipc['mes_año'], format='%Y%m').dt.year
ipc['mes'] = pd.to_datetime(ipc['mes_año'], format='%Y%m').dt.month
ipc = ipc[ipc['mes'] == 12]
ipc = ipc.filter(['indice', 'year']).reset_index(drop=True)
ipc = ipc.sort_values(by='year').reset_index(drop=True)
ipc['indice'] = ipc['indice'].str.replace(',','.')
ipc['indice'] = ipc['indice'].astype(float)
ipc['year'] += 1

In [17]:
gastos_ipc = df.merge(ipc)
gastos_ipc
gastos_ipc['cop_def'] = gastos_ipc['cop'] / (gastos_ipc['indice'] / 100)
gastos_ipc.loc[gastos_ipc['entidad'] == 'CLUB MILITAR DE OFICIALES', 'sector_code'] = 15
gastos_ipc['entidad'] = gastos_ipc['entidad'].str.strip()
gastos_ipc[gastos_ipc['cuenta']!='NACIONAL'].replace('INVERSIÓN','INVERSION').reset_index(drop=True).to_csv('data/gastos_2013_2024_const.csv', index=False)

JSON entidades

In [49]:
dictio = {}

for year, sector, entity_code, entidad in gastos_ipc.groupby(['year', 'sector_code', 'entity_code', 'entidad'])['cop'].sum().index:
  if dictio.get(year) == None:
    dictio[year] = {}
    if dictio[year].get(sector) == None:
      dictio[year][sector] = {}
      if dictio[year][sector].get(entity_code) == None:
        dictio[year][sector][entity_code] = entidad
      else:
        pass
    else:
      if dictio[year][sector].get(entity_code) == None:
        dictio[year][sector][entity_code] = entidad
      else:
        pass
  else:
    if dictio[year].get(sector) == None:
      dictio[year][sector] = {}
      if dictio[year][sector].get(entity_code) == None:
        dictio[year][sector][entity_code] = entidad
      else:
        pass
    else:
      if dictio[year][sector].get(entity_code) == None:
        dictio[year][sector][entity_code] = entidad
      else:
        pass

In [50]:
with open("data/dictio.json", 'w') as ds:
  json.dump(dictio, ds)

#EXP

In [51]:
gastos_ipc = gastos_ipc[gastos_ipc['cuenta']!='NACIONAL'].replace('INVERSIÓN','INVERSION').reset_index(drop=True)

In [52]:
gastos_ipc.groupby(['year'])[['cop', 'cop_def']].sum()

,cop,cop_def
year,,
2013,1.855246e+14,2.376997e+14
2014,2.030000e+14,2.551533e+14
2015,2.161581e+14,2.621051e+14
2016,2.159144e+14,2.452179e+14
2017,2.244217e+14,2.410285e+14
2018,2.355538e+14,2.430394e+14
2019,2.589973e+14,2.589973e+14
2020,2.717140e+14,2.617669e+14
2021,3.139980e+14,2.976849e+14
